In [1]:
import warnings

warnings.filterwarnings("ignore", category=ImportWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning, module="ipykernel")

In [3]:
import os
from os.path import join
import numpy as np
import pandas as pd
import random
import time
from scipy.stats import linregress

random.seed(4738)

warnings.filterwarnings("ignore", category=ImportWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning, module="ipykernel")

from pythermalcomfort.models import JOS3

/Users/hsuan/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


SystemError: initialization of _internal failed without raising an exception

In [ ]:
home = os.getcwd()
out = join(home, "output")
healthTable = pd.read_csv(join(home, "Health Data.csv"))
healthTable

Assume we have 100 users, so we will have 100 personal health data. And if we got 10 time points of weather data (e.g. per-30-mins data for 5 hours), we will have 100 × 30 data points of perceived temperature.

In [ ]:
def getSexCode(sex):

  if sex == "Male":
    return 0

  elif sex == "Female":
    return 1

  else:
    return random.randint(0,1)

In [ ]:
def getStrokeVolume(sexCode, age):

  if sexCode == 0:
    coef1 = np.polyfit([30, 50, 70], [39, 37, 35], 0)
    # coef2 = np.polyfit([30, 50, 70], [91, 83, 75], 0)
    coef2 = np.polyfit([30, 50, 70], [104, 95, 87], 0)

  else:
    coef1 = np.polyfit([30, 50, 70], [34, 30, 27], 0)
    # coef2 = np.polyfit([30, 50, 70], [72, 66, 60], 0)
    coef2 = np.polyfit([30, 50, 70], [84, 78, 72], 0)

  low_lm = np.poly1d(coef1)
  up_lm = np.poly1d(coef2)

  lowSV = low_lm(age)/1000
  upSV = up_lm(age)/1000

  midSV = 0.3*lowSV + 0.7*upSV

  return np.random.choice([random.uniform(lowSV, midSV), random.uniform(midSV, upSV)], 1, p = [0.4, 0.6])[0]

In [ ]:
def assumption_data_generator (N, sex):
  ### note: this is the draft framework of generating assumption data, could be amended.
  # N: number of data points
  # sex: the sex of the dataset
  sex = sex.title()
  sexCode = getSexCode(sex)
  chooseSex = True if sex == "Unknown" else False

  df = {"sex":[], "age":[], "height":[], "weight":[], "BMI":[], "freqOfExercise":[], "CI":[]}

  for i in range(N):

    # check if we should randomly pick a gender
    if chooseSex:
      sexCode = getSexCode(sex)
      sex = "Male" if sexCode == 0 else "Female"

    df["sex"].append(sexCode)

    # generate age depending on population
    if sex == 0:
      p_age = [0.09, 0.19, 0.17, 0.17, 0.16, 0.21]
      p_freq = [0.47, 0.23, 0.2, 0.1]
    else:
      p_age = [0.08, 0.19, 0.17, 0.17, 0.16, 0.23]
      p_freq = [0.66, 0.15, 0.12, 0.07]

    age = np.random.choice([random.randint(18,24), random.randint(25,34), random.randint(35,44), random.randint(45,54), random.randint(55,64), random.randint(65,86)], 1, p = p_age)[0]
    df["age"].append(age)

    # get the row id of the current data point
    rID = healthTable[(healthTable["Age"] <= age) & (healthTable["Sex"] == sex)].index[-1]

    # height
    height = round(np.random.normal(healthTable.loc[rID, "AvgHeight"], healthTable.loc[rID, "RSEHeight"]), 2)
    df["height"].append(height)

    # weight
    weight = round(np.random.normal(healthTable.loc[rID, "AvgWeight"], healthTable.loc[rID, "RSEWeight"]), 2)
    df["weight"].append(weight)

    # calculate the BMI
    df["BMI"].append(round(weight/((height/100)**2), 2))

    # generate the freq of exercise
    if age >= 80:
      freqExercise = random.randint(0,1)
    elif age >= 65:
      freqExercise = random.randint(0,2)
    else:
      freqExercise = np.random.choice([random.randint(0,1), random.randint(2,3), random.randint(4,5), random.randint(6,7)], 1, p = p_freq)[0]

    df["freqOfExercise"].append(freqExercise)

    ### calculation of Cardiac index
    # generate the Heart Rate
    freq_type = int(freqExercise/2) + 1
    lowHR = healthTable.loc[rID, "LowerHR"+str(freq_type)]
    upHR = healthTable.loc[rID, "UpperHR"+str(freq_type)]
    HR = random.randint(lowHR, upHR)

    # generate the Stroke Volume data
    SV = getStrokeVolume(sexCode, age)

    # calculate BSA
    BSA = np.sqrt(height*weight/3600)

    # calculate CI (normal range: 2.5~4.2 L/min/m^2)
    CI = (SV*HR) / BSA
    df["CI"].append(CI)

  return pd.DataFrame(df)

In [ ]:
data = pd.concat([assumption_data_generator(5, "Male"), assumption_data_generator(5, "Female")], ignore_index=True)
data.insert(0, "PersonID", range(1, len(data)+1))
data.to_csv(join(out, "example.csv"))
data.head()

TBD: how to generate data with unknown sex? One possible way is to randomly chose one of genders and gernate the data; the other is use the general distribution of height and weight.

In [ ]:
data

In [ ]:
data.describe(include="all")

The following part is for combining actual weather data and assumption data.

In [ ]:
morningWeather = pd.read_csv(join(home, "MorningWeatherDataset.csv"), index_col=0)
morningWeather

In [ ]:
afternoonWeather = pd.read_csv(join(home, "AfternoonWeatherDataset.csv"), index_col=0)
afternoonWeather

In [ ]:
MorningData = pd.merge(morningWeather, data, how="cross")
#MorningData.to_csv(join(out, "MorningDataset.csv"))
MorningData

In [ ]:
AfternoonData = pd.merge(afternoonWeather, data, how="cross")
#AfternoonData.to_csv(join(out, "AfternoonDataset.csv"))
AfternoonData

Testing 1 input default "feels like" vs the model to see if there is any difference

In [ ]:
def personalizedFeelLike(sex, age, height, weight, bmi, ci, tempC, wind, humidity):
  # sex: all lower case
  # age: in years
  # height: in metres
  # weight: in kg
  # ci: cardiac index L/min/m^2
  # temp: in Celcius
  # wind: in metres per hour
  # humidity: in %

  # For men, the formula is (1.20 x BMI) + (0.23 x Age) – 16.2, while for women, it is (1.20 x BMI) + (0.23 x Age) – 5.4.
  # Not sure what to use if the sex is "unknown"
  if sex == 0:
    percentageFat = (1.20 * bmi) + (0.23 * age) - 16.2
  else:
    percentageFat = (1.20 * bmi) + (0.23 * age) - 5.4

  height = height / 100
  wind = wind * 1000


  model = JOS3(
      height=height,
      weight=weight,
      fat=percentageFat,
      age=age,
      sex=sex,
      ci=ci
  )

  model.tdb = tempC
  model.v = wind
  model.rh = humidity

  model.simulate(
      times=5,  # Number of loops of a simulation
      dtime=20,  # Time delta [sec]. The default is 60.
  )  # Exposure time = 30 [loops] * 60 [sec] = 30 [min]

  df = pd.DataFrame(model.dict_results())
  # df
  return df["t_skin_mean"].iloc[-1]


def defaultFeelsLike(tempC, wind, humidity):
  tempF = (tempC * 9/5) + 32
  windMiles = wind/1609

  feelsLikeF = 35.74 + (0.6215*tempF) - 35.75*(windMiles**0.16) + ((0.4275*tempF * windMiles**0.16))
  feelsLikeC = (feelsLikeF - 32) * 5/9
  return feelsLikeC

Add difference between **perceived temperature** and **temperature** into the dataset.

In [ ]:
perceivedTemps = []
for i in range(len(MorningData)):
  row = MorningData.iloc[i,]
  sex = row["sex"]
  age = row["age"]
  height = row["height"]
  weight = row["weight"]
  bmi = row["BMI"]
  ci = row["CI"]
  tempC = row["Temp9am"]
  wind = row["WindSpeed9am"]
  humidity = row["Humidity9am"]

  perTemp = personalizedFeelLike(sex, age, height, weight, bmi, ci, tempC, wind, humidity)

  perceivedTemps.append(perTemp)

MorningData["perceivedTemp"] = perceivedTemps
MorningData.to_csv(join(out, "MorningDataset.csv"))

In [ ]:
perceivedTemps = []
for i in range(len(AfternoonData)):
  row = AfternoonData.iloc[i,]
  sex = row["sex"]
  age = row["age"]
  height = row["height"]
  weight = row["weight"]
  bmi = row["BMI"]
  ci = row["CI"]
  tempC = row["Temp3pm"]
  wind = row["WindSpeed3pm"]
  humidity = row["Humidity3pm"]

  perTemp = personalizedFeelLike(sex, age, height, weight, bmi, ci, tempC, wind, humidity)

  perceivedTemps.append(perTemp)

AfternoonData["perceivedTemp"] = perceivedTemps
AfternoonData.to_csv(join(out, "AfternoonDataset.csv"))

In [ ]:
AfternoonData["perceivedTemperature"] = AfternoonData.apply(lambda row : personalizedFeelLike(row["sex"], row["age"], row["height"], row["weight"], row["BMI"], row["CI"], row["Temp3pm"], row["WindSpeed3pm"], row["Humidity3pm"]), axis=1)
AfternoonData

In [ ]:
tempC = 20
wind = 20*1000 # in m/h
humidity = 64 # in %

defResults = defaultFeelsLike(tempC, wind, humidity)
print("Using default feels-like formula")
print(defResults)

sex = "male"
age = 86
height = 173.03
weight = 91.11
bmi = 30.43
ci = 2.4


josResults = personalizedFeelLike(sex, age, height, weight, bmi, ci, tempC, wind, humidity)
print("Using JOS-3")
print(josResults)


sex = "female"
age = 61
height = 160.69
weight = 74.6
bmi = 28.89
ci = 3.2

tempC = 20
wind = 20*1000 # in m/h
humidity = 64 # in %


josResults = personalizedFeelLike(sex, age, height, weight, bmi, ci, tempC, wind, humidity)
print("Using JOS-3")
print(josResults)

In [ ]:
tempC = 20
wind = 20*1000 # in m/h
humidity = 64 # in %

defResults = defaultFeelsLike(tempC, wind, humidity)
print("Using default feels-like formula")
print(defResults)

sex = "male"
age = 89
height = 173.03/100
weight = 91.11
bmi = 30.43
ci = 2.4


josResults = personalizedFeelLike(sex, age, height, weight, bmi, tempC, wind, humidity)
print("Using JOS-3")
print(josResults)


sex = "female"
age = 61
height = 160.69/100
weight = 74.6
bmi = 28.89

tempC = 20
wind = 20*1000 # in m/h
humidity = 64 # in %


josResults = personalizedFeelLike(sex, age, height, weight, bmi, tempC, wind, humidity)
print("Using JOS-3")
print(josResults)


sex = "female"
age = 19
height = 163.42/100
weight = 66.76
bmi = 25.0

tempC = 20
wind = 20*1000 # in m/h
humidity = 64 # in %


josResults = personalizedFeelLike(sex, age, height, weight, bmi, tempC, wind, humidity)
print("Using JOS-3")
print(josResults)



# need to also take into account frequency of exercise (by some coefficient?)